# Введение в NLP‑эмбеддинги на PyTorch: TF‑IDF и Word2Vec (Disaster Tweets)

В этом практическом ноутбуке по шагам разберём:
- что такое эмбеддинги текста и зачем они нужны;
- как посчитать TF‑IDF и интерпретировать веса;
- как обучить Word2Vec (CBOW и Skip‑Gram) на PyTorch без классов и функций — максимально линейный, читаемый код;
- как визуализировать векторные представления слов и документов c помощью Plotly (PCA/t‑SNE);
- как сравнить TF‑IDF и Word2Vec и сделать выводы.

Датасет: Kaggle `Disaster Tweets` (`vstepanenko/disaster-tweets`). Если локально есть `tweets.csv`, используем его. В противном случае — инструкция по ручной загрузке с Kaggle.

Полезные ссылки:
- Страница датасета — `https://www.kaggle.com/datasets/vstepanenko/disaster-tweets`
- TF‑IDF (sklearn) — `https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction`
- Word Embeddings Tutorial (PyTorch) — `https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html`
- Word2Vec (Mikolov et al.) — `https://arxiv.org/abs/1301.3781`, `https://arxiv.org/abs/1310.4546`
- Plotly — `https://plotly.com/python/`



In [ ]:
# Установка/импорты, сиды, device
import os, random, re, string, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import SGD

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import plotly.express as px
from collections import Counter

# Сиды для воспроизводимости
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

pd.set_option('display.max_colwidth', 120)



## Датасеты и выбор Disaster Tweets

`Disaster Tweets` — короткие тексты твитов с бинарной меткой `target` о наличии признаков катастрофы. Формат удобен для построения базовых представлений (TF‑IDF, Word2Vec) и интерактивной визуализации.

Другие корпуса:
- IMDB (рецензии, бинарная сентимент‑классификация)
- SMS Spam (спам/не‑спам)
- AG News (4‑классовые новости)
- BBC News (тематики новостей)
- TREC‑6 (тип вопроса)

Для воспроизведения требуется таблица со столбцами `text`, `target`. При отсутствии `tweets.csv` загрузите с Kaggle: `https://www.kaggle.com/datasets/vstepanenko/disaster-tweets`


In [ ]:
# Загрузка датасета: пытаемся прочитать локальный tweets.csv, иначе даём подсказку
csv_path = "tweets.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    print("Файл tweets.csv не найден. Скачайте с Kaggle и поместите в корень проекта:")
    print("https://www.kaggle.com/datasets/vstepanenko/disaster-tweets")
    print("Ожидаемые столбцы: text, target")
    # создаём маленький демо‑датасет для отладки структуры ноутбука
    df = pd.DataFrame({
        "text": [
            "Fire in the street near river.",
            "Good morning everyone!",
            "Earthquake reported yesterday.",
            "I love sunny days and coffee.",
            "Flood alert in the city center!"
        ],
        "target": [1, 0, 1, 0, 1]
    })

df = df[["text", "target"]].dropna()
df.head()


## Предобработка текста

Шаги:
- приведение к нижнему регистру;
- замена ссылок на токен `<LINK>` и упоминаний на `<MENTION>`;
- удаление пунктуации и цифр; сжатие пробелов;
- токенизация через `.split()` (при необходимости можно заменить на `nltk`).


In [ ]:
# регулярные выражения для токенов ссылок и упоминаний
url_re = re.compile(r"https?://\S+|www\.\S+")
mention_re = re.compile(r"@[A-Za-z0-9_]+")
# хэштеги не удаляются отдельной регул. замены; символ # будет удалён при удалении пунктуации
number_re = re.compile(r"\d+")
punct_table = str.maketrans("", "", string.punctuation)

# исходные тексты и целевые метки
texts = df["text"].astype(str).tolist()
labels = df["target"].astype(int).tolist()

# очистка и нормализация текста
clean_texts = []
for t in texts:
    t = t.lower()                                              # нижний регистр
    t = url_re.sub(" <LINK> ", t)                             # ссылки → токен <LINK>
    t = mention_re.sub(" <MENTION> ", t)                      # упоминания → токен <MENTION>
    t = number_re.sub(" ", t)                                 # удаление цифр
    t = t.translate(punct_table)                                # удаление пунктуации
    t = re.sub(r"\s+", " ", t).strip()                      # схлопывание пробелов
    clean_texts.append(t)

# токенизация по пробелам
tokenized = [t.split() for t in clean_texts]

# разбиение на обучающую/валидационную части
train_texts, valid_texts, train_labels, valid_labels = train_test_split(
    clean_texts, labels, test_size=0.2, random_state=seed, stratify=labels if len(set(labels))>1 else None
)

# контрольный вывод
print("Пример очищенного текста:", clean_texts[0])
print("Размеры:", len(train_texts), len(valid_texts))


## TF‑IDF: идея и формулы

TF — частота термина в документе. IDF — обратная документная частота, уменьшающая вес часто встречающихся слов.

Формулы (с учётом сглаживания в sklearn):
- tf — относительная частота в документе;
- idf = log((N + 1) / (df + 1)) + 1, где N — число документов, df — в скольких документах слово встречается;
- tfidf = tf * idf.

Далее посчитаем TF‑IDF для обучающей части корпуса, преобразуем в `torch.tensor` и покажем простые операции (нормы, косинусная близость).


In [ ]:
max_features = 3000
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=(1,1))
X_train = vectorizer.fit_transform(train_texts)
X_valid = vectorizer.transform(valid_texts)

print("TF‑IDF форма:", X_train.shape)

# Перевод в torch
tf_train = torch.tensor(X_train.toarray(), dtype=torch.float32, device=device)
tf_valid = torch.tensor(X_valid.toarray(), dtype=torch.float32, device=device)

# Косинусная близость между первыми двумя документами
u = tf_train[0]
v = tf_train[1] if tf_train.shape[0] > 1 else tf_train[0]
cos_sim = torch.nn.functional.cosine_similarity(u.unsqueeze(0), v.unsqueeze(0)).item()
print("Косинусная близость первых двух документов:", round(cos_sim, 4))

vocab_tfidf = vectorizer.get_feature_names_out()



In [ ]:
# Визуализации TF‑IDF: документы в 2D (PCA) и топ‑слова

# Документы в 2D по PCA
pca_docs = PCA(n_components=2, random_state=seed)
coords = pca_docs.fit_transform(X_train.toarray())
fig = px.scatter(x=coords[:,0], y=coords[:,1], color=pd.Series(train_labels, name="target"),
                 title="Документы (TF‑IDF) в проекции PCA")
fig.show()

# Топ‑слова по среднему TF‑IDF в положительном/отрицательном классе
train_df = pd.DataFrame({"text": train_texts, "target": train_labels})
X_all = vectorizer.transform(train_df["text"]).toarray()
mean_pos = X_all[np.array(train_df["target"])==1].mean(axis=0) if (np.array(train_df["target"])==1).any() else np.zeros(X_all.shape[1])
mean_neg = X_all[np.array(train_df["target"])==0].mean(axis=0) if (np.array(train_df["target"])==0).any() else np.zeros(X_all.shape[1])

k = 15
top_pos_idx = np.argsort(-mean_pos)[:k]
top_neg_idx = np.argsort(-mean_neg)[:k]

fig1 = px.bar(x=vocab_tfidf[top_pos_idx], y=mean_pos[top_pos_idx], title="Топ‑слова TF‑IDF: target=1")
fig2 = px.bar(x=vocab_tfidf[top_neg_idx], y=mean_neg[top_neg_idx], title="Топ‑слова TF‑IDF: target=0")
fig1.show(); fig2.show()


## Подготовка словаря и данных для Word2Vec (CBOW/Skip‑Gram)

Сделаем простой словарь по частотам, добавим `PAD` и `UNK`, отфильтруем редкие слова. Затем создадим пары для CBOW и Skip‑Gram одним линейным кодом с окном контекста.


In [ ]:
# Строим словарь и индексируем тексты
min_freq = 2
counter = Counter(w for t in tokenized for w in t)
# базовые токены
itos = ["<PAD>", "<UNK>"] + [w for w, c in counter.items() if c >= min_freq]
stoi = {w: i for i, w in enumerate(itos)}

# индексируем тексты
token_ids = []
for t in tokenized:
    ids = [stoi.get(w, 1) for w in t]  # 1 == <UNK>
    if len(ids) > 0:
        token_ids.append(ids)

# параметры словаря и размер эмбеддинга
vocab_size = len(itos)
emb_dim = 100
print("Словарь:", vocab_size, "Документов:", len(token_ids))


In [ ]:
# Генерация пар для CBOW и Skip‑Gram
from tqdm.auto import tqdm

window = 2
cbow_pairs = []   # (context_ids, target_id)
skip_pairs = []   # (target_id, context_id)
for ids in tqdm(token_ids, desc="Генерация пар"):
    L = len(ids)
    for i in range(L):
        left = max(0, i - window)
        right = min(L, i + window + 1)
        context = [ids[j] for j in range(left, right) if j != i]
        if len(context) == 0:
            continue
        cbow_pairs.append((context, ids[i]))
        for c in context:
            skip_pairs.append((ids[i], c))

# Ограничим количество для демонстрации
max_pairs = 5000
cbow_pairs = cbow_pairs[:max_pairs]
skip_pairs = skip_pairs[:max_pairs]

print("CBOW пар:", len(cbow_pairs), "SG пар:", len(skip_pairs))


## Word2Vec CBOW (полный софтмакс)

Идея: по контексту предсказываем центральное слово. Архитектура:
- `Embedding(vocab_size, emb_dim)` для слов;
- усреднение контекстных векторов;
- `Linear(emb_dim → vocab_size)` и `CrossEntropyLoss` (эквивалентно `LogSoftmax + NLLLoss`).

Функции активации и потерь:
- На выходе линейного слоя получаем логиты. `CrossEntropyLoss` внутри применяет `LogSoftmax`, поэтому вручную `Softmax` не нужен.
- Интерпретация: `Softmax` превращает логиты в вероятности по словарю; `CrossEntropy` наказывает за неправильный индекс целевого слова.



### Что такое `nn.Embedding` в PyTorch

`nn.Embedding` — это обучаемая таблица соответствий индексов и векторов признаков (lookup table).
Фактически это матрица весов `W ∈ R[num_embeddings × embedding_dim]`, где каждая строка — вектор слова/токена.
Входом служат целочисленные индексы (`LongTensor`) формы `[..., sequence_len]`, на выходе — вещественные векторы формы `[..., sequence_len, embedding_dim]`.

Ключевые параметры:
- `num_embeddings`: размер словаря (число токенов);
- `embedding_dim`: размерность эмбеддинга;
- `padding_idx` (опц.): индекс токена‑паддинга; соответствующая строка не обучается и всегда остаётся нулевой;
- `max_norm`, `norm_type` (опц.): ограничение нормы строк при обновлении;
- `scale_grad_by_freq` (опц.): масштабирование градиента по частоте слова;
- `sparse` (опц.): разрежённые градиенты для ускорения при больших словарях.

Как это работает:
- Эквивалент умножению one‑hot представления на матрицу `W`, но без явного создания one‑hot; извлекаются нужные строки `W` по индексам.
- Градиенты протекают только через те строки, которые были использованы на данном батче.
- Слой не применяет смещения или нелинейности: это чистый lookup, поэтому в CBOW/Skip‑Gram поверх усреднённого/взятого эмбеддинга добавляется `Linear` и уже затем считается `CrossEntropyLoss`.

В этом ноутбуке:
- В CBOW берём эмбеддинги контекстных слов и усредняем; результат подаётся в `Linear(embedding_dim → vocab_size)`.
- В Skip‑Gram эмбеддинг центрального слова подаётся в `Linear(embedding_dim → vocab_size)` для предсказания контекстного слова.


In [ ]:
# Простой CBOW без классов: просто объявляем слои и обучаем
from tqdm.auto import tqdm

cbow_embed = nn.Embedding(vocab_size, emb_dim).to(device)
cbow_linear = nn.Linear(emb_dim, vocab_size).to(device)

cbow_loss = nn.CrossEntropyLoss()
cbow_opt = SGD(list(cbow_embed.parameters()) + list(cbow_linear.parameters()), lr=0.1)

epochs = 3
for epoch in range(epochs):
    total_loss = 0.0
    for ctx_ids, tgt_id in tqdm(cbow_pairs, desc=f"CBOW epoch {epoch+1}"):
        ctx_tensor = torch.tensor(ctx_ids, dtype=torch.long, device=device)
        tgt_tensor = torch.tensor([tgt_id], dtype=torch.long, device=device)

        # прямой проход
        ctx_vecs = cbow_embed(ctx_tensor)          # [context_len, emb_dim]
        ctx_mean = ctx_vecs.mean(dim=0, keepdim=True)  # [1, emb_dim]
        logits = cbow_linear(ctx_mean)             # [1, vocab_size]

        loss = cbow_loss(logits, tgt_tensor)

        cbow_opt.zero_grad()
        loss.backward()
        cbow_opt.step()

        total_loss += loss.item()
    print(f"CBOW epoch {epoch+1}: loss={total_loss:.2f}")

# Сохраняем обученные эмбеддинги слов (матрица)
embeddings_cbow = cbow_embed.weight.detach().cpu().numpy()



## Word2Vec Skip‑Gram (полный софтмакс)

Идея: по центральному слову предсказываем каждое слово из его контекста. Архитектура аналогична:
- `Embedding(vocab_size, emb_dim)`;
- `Linear(emb_dim → vocab_size)`;
- `CrossEntropyLoss`.

Замечание: для ускорения в реальных проектах применяют Negative Sampling и `BCEWithLogitsLoss`, но здесь используем полный софтмакс ради простоты и прозрачности.


In [ ]:
# Простой Skip‑Gram без классов
sg_embed = nn.Embedding(vocab_size, emb_dim).to(device)
sg_linear = nn.Linear(emb_dim, vocab_size).to(device)

sg_loss = nn.CrossEntropyLoss()
sg_opt = SGD(list(sg_embed.parameters()) + list(sg_linear.parameters()), lr=0.1)

epochs = 3
for epoch in range(epochs):
    total_loss = 0.0
    for tgt_id, ctx_id in skip_pairs:
        tgt_tensor = torch.tensor([tgt_id], dtype=torch.long, device=device)
        ctx_tensor = torch.tensor([ctx_id], dtype=torch.long, device=device)

        tgt_vec = sg_embed(tgt_tensor)      # [1, emb_dim]
        logits = sg_linear(tgt_vec)         # [1, vocab_size]

        loss = sg_loss(logits, ctx_tensor)

        sg_opt.zero_grad()
        loss.backward()
        sg_opt.step()

        total_loss += loss.item()
    print(f"Skip‑Gram epoch {epoch+1}: loss={total_loss:.2f}")

embeddings_sg = sg_embed.weight.detach().cpu().numpy()



## Визуализация эмбеддингов слов (PCA / t‑SNE)

Сравним распределения слов для CBOW и Skip‑Gram. Для наглядности возьмём топ‑N слов по частоте.


In [ ]:
# Выберем топ‑N частых слов (исключая спец‑токены)
N = 300
freq_sorted = sorted([(w, c) for w, c in counter.items() if w in stoi], key=lambda x: -x[1])
words = [w for w, _ in freq_sorted if w not in ("<PAD>", "<UNK>")][:N]
idxs = [stoi[w] for w in words]

sub_cbow = embeddings_cbow[idxs]
sub_sg = embeddings_sg[idxs]

# PCA до 2D
pca2 = PCA(n_components=2, random_state=seed)
coords_cbow = pca2.fit_transform(sub_cbow)
coords_sg = pca2.fit_transform(sub_sg)

fig = px.scatter(x=coords_cbow[:,0], y=coords_cbow[:,1], text=words, title="CBOW: слова в 2D (PCA)")
fig.update_traces(textposition='top center', marker=dict(size=6))
fig.show()

fig = px.scatter(x=coords_sg[:,0], y=coords_sg[:,1], text=words, title="Skip‑Gram: слова в 2D (PCA)")
fig.update_traces(textposition='top center', marker=dict(size=6))
fig.show()


## Ближайшие слова по косинусной близости

Найдём ближайшие слова для нескольких запросов (например: `fire`, `earthquake`, `flood`, `help`) по эмбеддингам CBOW и Skip‑Gram.


In [ ]:
def nearest_words(query_word, emb_matrix, k=8):
    if query_word not in stoi:
        return []
    q_idx = stoi[query_word]
    qv = emb_matrix[q_idx]
    M = emb_matrix
    # косинусная близость вручную
    num = (M @ qv)
    denom = (np.linalg.norm(M, axis=1) * (np.linalg.norm(qv) + 1e-9) + 1e-9)
    cos = num / denom
    order = np.argsort(-cos)
    result = []
    for idx in order[:k+1]:  # +1 чтобы включить сам запрос, потом отфильтруем
        w = itos[idx] if idx < len(itos) else None
        if w is None or w in ("<PAD>", "<UNK>") or w == query_word:
            continue
        result.append((w, float(cos[idx])))
        if len(result) == k:
            break
    return result

for word in ["fire", "earthquake", "flood", "help"]:
    print("\nСлово:", word)
    print("CBOW:", nearest_words(word, embeddings_cbow, k=8))
    print("SG:  ", nearest_words(word, embeddings_sg, k=8))


## Документные эмбеддинги и сравнение с TF‑IDF

Простой способ получить вектор документа на базе словарных эмбеддингов — среднее по словам (можно также взвешивать по TF‑IDF). Сравним распределения документов, полученных по CBOW/SG, с проекцией TF‑IDF.


In [ ]:
# усреднение эмбеддингов слов в документе

# подготовим индексированные документы для train_texts (как после очистки)
train_tokens = [t.split() for t in train_texts]
train_ids = [[stoi.get(w, 1) for w in doc if w in stoi] for doc in train_tokens]

# среднее по CBOW
doc_emb_cbow = []
for ids in train_ids:
    if len(ids) == 0:
        doc_emb_cbow.append(np.zeros(emb_dim))
    else:
        doc_emb_cbow.append(embeddings_cbow[ids].mean(axis=0))
doc_emb_cbow = np.stack(doc_emb_cbow)

# среднее по SG
doc_emb_sg = []
for ids in train_ids:
    if len(ids) == 0:
        doc_emb_sg.append(np.zeros(emb_dim))
    else:
        doc_emb_sg.append(embeddings_sg[ids].mean(axis=0))
doc_emb_sg = np.stack(doc_emb_sg)

# PCA до 2D и визуализация
pca_docs2 = PCA(n_components=2, random_state=seed)
coords_cbow_docs = pca_docs2.fit_transform(doc_emb_cbow)
fig = px.scatter(x=coords_cbow_docs[:,0], y=coords_cbow_docs[:,1], color=pd.Series(train_labels, name="target"),
                 title="Документы: среднее словарных эмбеддингов (CBOW) → PCA")
fig.show()

coords_sg_docs = pca_docs2.fit_transform(doc_emb_sg)
fig = px.scatter(x=coords_sg_docs[:,0], y=coords_sg_docs[:,1], color=pd.Series(train_labels, name="target"),
                 title="Документы: среднее словарных эмбеддингов (SG) → PCA")
fig.show()


## Выводы

- **TF‑IDF** хорошо работает как базовый метод: просто, быстро, интерпретируемо. Он учитывает важность слов через IDF, но не понимает семантику и порядок слов.
- **Word2Vec (CBOW/Skip‑Gram)** учится на контекстах и лучше улавливает семантическую близость слов. CBOW обычно быстрее и устойчивее на маленьких данных, Skip‑Gram — лучше для редких слов.
- **Визуализации** помогают увидеть кластеры и интуитивно понять структуру пространства эмбеддингов.
- Для продвинутых задач: попробуйте `negative sampling` (BCEWithLogitsLoss), `fastText` (учёт субслов), предобученные векторы (`GloVe`), трансформеры (`BERT` и др.).

Полезные ссылки:
- Kaggle Disaster Tweets — `https://www.kaggle.com/datasets/vstepanenko/disaster-tweets`
- PyTorch Word Embeddings Tutorial — `https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html`
- TF‑IDF (sklearn) — `https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction`
- Word2Vec: Mikolov et al. — `https://arxiv.org/abs/1301.3781`, `https://arxiv.org/abs/1310.4546`
- Plotly — `https://plotly.com/python/`


## Как это будет на предобученных эмбеддингах (EmbeddingGemma‑300M)

Используются эмбеддинги из модели `google/embeddinggemma-300m` (Hugging Face). Размер вектора по умолчанию — 768. Для демонстрации: извлечение эмбеддингов для отдельных твитов, визуализация в 2D и поиск ближайших соседей.

Ссылки:
- Модель: `https://huggingface.co/google/embeddinggemma-300m`


In [ ]:
#!uv add sentence-transformers hf_xet
!pip install sentence-transformers hf_xet

In [ ]:
# установка и импорт модели эмбеддингов
# !pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

# загрузка предобученной модели (использует float32/bfloat16)
embed_model = SentenceTransformer("google/embeddinggemma-300m", device=str(device))

# --- Визуализация эмбеддингов отдельных слов (топ-N по частоте) ---
N_words = 200
words_sorted = sorted(counter.items(), key=lambda x: -x[1])
words_for_gemma = [w for w, c in words_sorted if w and w not in ("<PAD>", "<UNK>")][:N_words]

emb_words = embed_model.encode(words_for_gemma, convert_to_tensor=True, device=str(device))
emb_words = emb_words.detach().cpu().numpy()

pca_w = PCA(n_components=2, random_state=seed)
coords_w = pca_w.fit_transform(emb_words)
df_words = pd.DataFrame({"x": coords_w[:,0], "y": coords_w[:,1], "word": words_for_gemma})
fig = px.scatter(df_words, x="x", y="y", text="word", hover_name="word", title="EmbeddingGemma‑300M: слова → PCA")
fig.update_traces(textposition="top center", marker=dict(size=6))
fig.show()

# --- Документные эмбеддинги с подсказкой исходного текста ---
# выбор подмножества текстов для демонстрации
sample_texts = train_texts[:500] if len(train_texts) > 500 else train_texts

# извлечение эмбеддингов документов
# Важно: использовать encode_document для документов, encode_query для запросов (если нужно сравнение запрос‑документ)
emb_gemma = embed_model.encode(sample_texts, convert_to_tensor=True, device=str(device))
emb_gemma = emb_gemma.detach().cpu().numpy()

# визуализация в 2D с подсказкой-оригиналом
pca_g = PCA(n_components=2, random_state=seed)
coords_g = pca_g.fit_transform(emb_gemma)
df_docs = pd.DataFrame({
    "x": coords_g[:,0],
    "y": coords_g[:,1],
    "target": pd.Series(train_labels[:len(sample_texts)], name="target"),
    "text": sample_texts
})
fig = px.scatter(df_docs, x="x", y="y", color="target", hover_name="text", title="Документы: EmbeddingGemma‑300M → PCA")
fig.show()

# пример ближайших соседей среди документов по косинусной близости
from sklearn.metrics.pairwise import cosine_similarity
if len(sample_texts) >= 5:
    sims = cosine_similarity(emb_gemma)
    anchor = 0
    order = np.argsort(-sims[anchor])
    print("Пример документа:", sample_texts[anchor][:200])
    print("Ближайшие документы:")
    for idx in order[1:6]:
        print("\t", idx, round(float(sims[anchor, idx]), 3), sample_texts[idx][:200])


## Классификация текста на документах: сравнение представлений (PyTorch)

Сравниваются три представления документа:
- среднее словарных эмбеддингов CBOW;
- среднее словарных эмбеддингов Skip‑Gram;
- EmbeddingGemma‑300M документные эмбеддинги (классификация в отдельной ячейке ниже).

Модель классификации: линейный слой (`Linear(d, 1)`) и `BCEWithLogitsLoss`.
- Логиты преобразуются в вероятность через `Sigmoid` на инференсе;
- Порог классификации 0.5;
- Оптимизатор `Adam`;
- Прогресс тренировки — `tqdm`.

Метрики: точность и матрица ошибок (confusion matrix) на валидации.


In [ ]:
# подготовка документных эмбеддингов для train/valid
# TF‑IDF уже разделён; здесь считаем на Word2Vec

# CBOW среднее
def doc_mean_from_ids(ids_list, emb_matrix, emb_dim):
    out = []
    for ids in ids_list:
        if len(ids) == 0:
            out.append(np.zeros(emb_dim))
        else:
            out.append(emb_matrix[ids].mean(axis=0))
    return np.stack(out)

# индексы для train/valid
train_tokens = [t.split() for t in train_texts]
valid_tokens = [t.split() for t in valid_texts]
train_ids = [[stoi.get(w, 1) for w in doc if w in stoi] for doc in train_tokens]
valid_ids = [[stoi.get(w, 1) for w in doc if w in stoi] for doc in valid_tokens]

# документные векторы на основе средних словарных эмбеддингов
doc_cbow_train = doc_mean_from_ids(train_ids, embeddings_cbow, emb_dim)
doc_cbow_valid = doc_mean_from_ids(valid_ids, embeddings_cbow, emb_dim)

doc_sg_train = doc_mean_from_ids(train_ids, embeddings_sg, emb_dim)
doc_sg_valid = doc_mean_from_ids(valid_ids, embeddings_sg, emb_dim)

# --- Классификация на PyTorch (Linear + BCEWithLogitsLoss) ---
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# подготовка тензоров
y_train_t = torch.tensor(train_labels, dtype=torch.float32, device=device).unsqueeze(1)
y_valid_t = torch.tensor(valid_labels, dtype=torch.float32, device=device).unsqueeze(1)

# параметры обучения
batch_size = 64
epochs = 5
lr = 1e-3

# 1) CBOW документные векторы
Xtr = torch.tensor(doc_cbow_train, dtype=torch.float32, device=device)
Xva = torch.tensor(doc_cbow_valid, dtype=torch.float32, device=device)
model_cbow = nn.Linear(Xtr.shape[1], 1).to(device)
opt_cbow = torch.optim.Adam(model_cbow.parameters(), lr=lr)
crit = nn.BCEWithLogitsLoss()
for epoch in range(epochs):
    model_cbow.train()
    total_loss = 0.0
    for start in tqdm(range(0, Xtr.shape[0], batch_size), desc=f"CBOW epoch {epoch+1}"):
        end = start + batch_size
        xb = Xtr[start:end]
        yb = y_train_t[start:end]
        logits = model_cbow(xb)
        loss = crit(logits, yb)
        opt_cbow.zero_grad()
        loss.backward()
        opt_cbow.step()
        total_loss += float(loss.item())
# валидация
model_cbow.eval()
with torch.no_grad():
    val_logits = model_cbow(Xva)
    val_prob = torch.sigmoid(val_logits)
    val_pred = (val_prob >= 0.5).float()
    acc_cbow = float((val_pred.eq(y_valid_t)).float().mean().item())
print("CBOW val acc:", round(acc_cbow, 3))

# отчёт и матрица ошибок (CBOW)
y_true = y_valid_t.detach().cpu().numpy().ravel().astype(int)
y_hat = val_pred.detach().cpu().numpy().ravel().astype(int)
print("CBOW confusion matrix:\n", confusion_matrix(y_true, y_hat))
print("CBOW classification report:\n", classification_report(y_true, y_hat, digits=3))

# 2) Skip‑Gram документные векторы
Xtr = torch.tensor(doc_sg_train, dtype=torch.float32, device=device)
Xva = torch.tensor(doc_sg_valid, dtype=torch.float32, device=device)
model_sg = nn.Linear(Xtr.shape[1], 1).to(device)
opt_sg = torch.optim.Adam(model_sg.parameters(), lr=lr)
for epoch in range(epochs):
    model_sg.train()
    total_loss = 0.0
    for start in tqdm(range(0, Xtr.shape[0], batch_size), desc=f"SG epoch {epoch+1}"):
        end = start + batch_size
        xb = Xtr[start:end]
        yb = y_train_t[start:end]
        logits = model_sg(xb)
        loss = crit(logits, yb)
        opt_sg.zero_grad()
        loss.backward()
        opt_sg.step()
        total_loss += float(loss.item())
model_sg.eval()
with torch.no_grad():
    val_logits = model_sg(Xva)
    val_prob = torch.sigmoid(val_logits)
    val_pred = (val_prob >= 0.5).float()
    acc_sg = float((val_pred.eq(y_valid_t)).float().mean().item())
print("SG val acc:", round(acc_sg, 3))

# отчёт и матрица ошибок (SG)
y_true = y_valid_t.detach().cpu().numpy().ravel().astype(int)
y_hat = val_pred.detach().cpu().numpy().ravel().astype(int)
print("SG confusion matrix:\n", confusion_matrix(y_true, y_hat))
print("SG classification report:\n", classification_report(y_true, y_hat, digits=3))

print({"CBOW": round(acc_cbow, 3), "SkipGram": round(acc_sg, 3)})


## Классификация на EmbeddingGemma‑300M (PyTorch)

Обучение линейного классификатора на документных эмбеддингах EmbeddingGemma‑300M (без функций/классов), `BCEWithLogitsLoss`, `Adam`, `tqdm`. Вывод: точность и матрица ошибок.

Ссылка на модель: `https://huggingface.co/google/embeddinggemma-300m`


In [ ]:
# Документные эмбеддинги EmbeddingGemma (train/valid)
emb_gemma_train = embed_model.encode(train_texts, convert_to_tensor=True, device=str(device)).detach().cpu().numpy()
emb_gemma_valid = embed_model.encode(valid_texts, convert_to_tensor=True, device=str(device)).detach().cpu().numpy()

# Классификация на PyTorch
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report

Xtr = torch.tensor(emb_gemma_train, dtype=torch.float32, device=device)
Xva = torch.tensor(emb_gemma_valid, dtype=torch.float32, device=device)
y_train_t = torch.tensor(train_labels, dtype=torch.float32, device=device).unsqueeze(1)
y_valid_t = torch.tensor(valid_labels, dtype=torch.float32, device=device).unsqueeze(1)

model_g = nn.Linear(Xtr.shape[1], 1).to(device)
opt_g = torch.optim.Adam(model_g.parameters(), lr=1e-3)
crit = nn.BCEWithLogitsLoss()

epochs = 5
batch_size = 64
for epoch in range(epochs):
    model_g.train()
    total_loss = 0.0
    for start in tqdm(range(0, Xtr.shape[0], batch_size), desc=f"Gemma epoch {epoch+1}"):
        end = start + batch_size
        xb = Xtr[start:end]
        yb = y_train_t[start:end]
        logits = model_g(xb)
        loss = crit(logits, yb)
        opt_g.zero_grad()
        loss.backward()
        opt_g.step()
        total_loss += float(loss.item())

model_g.eval()
with torch.no_grad():
    val_logits = model_g(Xva)
    val_prob = torch.sigmoid(val_logits)
    val_pred = (val_prob >= 0.5).float()
    acc_g = float((val_pred.eq(y_valid_t)).float().mean().item())
print("Gemma val acc:", round(acc_g, 3))

# отчёт и матрица ошибок (Gemma)
y_true = y_valid_t.detach().cpu().numpy().ravel().astype(int)
y_hat = val_pred.detach().cpu().numpy().ravel().astype(int)
print("Gemma confusion matrix:\n", confusion_matrix(y_true, y_hat))
print("Gemma classification report:\n", classification_report(y_true, y_hat, digits=3))
